#### Notes:


<p>In this notebook, I will attempt to complete Exercise 2. It is intended to be readable by my teammates.</p> <br>

<u>Exercise 2:</u>

<ol>
    <li>Implement RLS and (1+1) EA introduced in w3</li>
    <li>Run Random Search, RLS and (1+1) EA 10 times on the problems below.</li>
</ol>


<u> Problems </u>

\begin{equation}
    \begin{aligned}
    \text{F1 OM: } \{0,1\}^n \to [0..n], x \mapsto \sum_{i=1}^{n}x_i \\ 

    \text{F3 LO: } \{0,1\}^n \to [0..n], x \mapsto \max{i \in [0..n] | \forall j\leq i: x_j = 1} = \sum_{i=1}^{n}\prod_{j=1}^{i} x_j \\ 

    \text{LHW: } f:\{0,1\}^n\to \R, x \mapsto \sum_{i}ix_i

    \text{LABS: }  x\mapsto \frac{n^2}{2\sum_{k=1}^{n-1}(\sum_{i=1}^{n-k}s_i s_{i+k})^2}, \text{where } s_i = 2x_i - 1. \\
    
    \text{N-Queens Problem}\\

    \text{Concentrated Trap}\\

    \text{NK Landscapes (NKL)}\\
    \end{aligned}
\end{equation}


### **<u>Terminologies</u>**


- `n`: (int) synonymous with the dimension of the problem or the problem size.
- `budget`: (int) or the number of iterations or function evalutations. 

In [26]:
import os 
import numpy as np 
from shutil import rmtree 
import glob 

In [27]:
def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)


def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)


def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())


clean()

In [28]:
import ioh 
from ioh import ProblemClass, logger 

In [29]:
#  a list of problem can be accessed via the base classes 
real_problems: dict[int, str] = ioh.problem.RealSingleObjective.problems 
print(real_problems)

{1: 'Sphere', 2: 'Ellipsoid', 3: 'Rastrigin', 4: 'BuecheRastrigin', 5: 'LinearSlope', 6: 'AttractiveSector', 7: 'StepEllipsoid', 8: 'Rosenbrock', 9: 'RosenbrockRotated', 10: 'EllipsoidRotated', 11: 'Discus', 12: 'BentCigar', 13: 'SharpRidge', 14: 'DifferentPowers', 15: 'RastriginRotated', 16: 'Weierstrass', 17: 'Schaffers10', 18: 'Schaffers1000', 19: 'GriewankRosenbrock', 20: 'Schwefel', 21: 'Gallagher101', 22: 'Gallagher21', 23: 'Katsuura', 24: 'LunacekBiRastrigin', 30: 'UniformStarDiscrepancy10', 31: 'UniformStarDiscrepancy25', 32: 'UniformStarDiscrepancy50', 33: 'UniformStarDiscrepancy100', 34: 'UniformStarDiscrepancy150', 35: 'UniformStarDiscrepancy200', 36: 'UniformStarDiscrepancy250', 37: 'UniformStarDiscrepancy500', 38: 'UniformStarDiscrepancy750', 39: 'UniformStarDiscrepancy1000', 40: 'SobolStarDiscrepancy10', 41: 'SobolStarDiscrepancy25', 42: 'SobolStarDiscrepancy50', 43: 'SobolStarDiscrepancy100', 44: 'SobolStarDiscrepancy150', 45: 'SobolStarDiscrepancy200', 46: 'SobolStarDis

In [30]:
def compute_mean(X: np.ndarray) -> float:
    return float(np.mean(X))


def compute_sd(X: np.ndarray) -> float:
    return float(np.std(X))

In [43]:
def run_experiment(problem: ioh.problem.PBO, algorithm, num_runs = 10):
    """
    Run an experiment of a given algorithm on a given problem for a number of runs (budget)
    - `problem`: an ioh problem instance
    - `algorithm`: a callable that takes a problem as input and runs the algorithm on it
    - `num_runs`: number of independent runs to perform
    """
    # For a known problem 18 w/ 32 variables, we know optimum = 8
    if problem.meta_data.problem_id == 18 and problem.meta_data.n_variables == 32:
        optimum = 8
    else:
        optimum = problem.optimum.y # otherwise, calculate optimum???
    print(optimum)
    # =============================================================


    for run in range(num_runs):
        # run the algorithm on the problem 
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found: {problem.state.current_best.y: .3f}")

        # reset the problem 
        problem.reset()

In [44]:
# random search class 
class RandomSearch:
    def __init__(self, budget: int) -> None:
        """
        - `budget`: (int) number of function evaluations to perform
        """
        self.budget = budget
    
    def __call__(self, problem_func: ioh.problem.PBO) -> None:
        """
        - `problem`: an ioh problem instance
        """
        for _ in range(self.budget):
            # sample a random solution 
            X: np.ndarray = np.random.randint(2, size=problem_func.meta_data.n_variables) # randomized binary solution: a vector of 0s and 1s of length n_variables 
            # evaluate the solution 
            problem_func(X.tolist()) # unfortunately, ioh does not accept numpy arrays as input so we need to convert it to a list

        return

#### Gather the required problems 

In [45]:
n = problem_size = 100 

In [46]:
p1 = ioh.get_problem(fid=1, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # Sphere
# grab problem 2 
p2 = ioh.get_problem(fid=2, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # 
# grab problem 3 
p3 = ioh.get_problem(fid=3, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #
# grab problem 18 
p18 = ioh.get_problem(fid=18, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p23 = ioh.get_problem(fid=23, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p24 = ioh.get_problem(fid=24, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 24
p24 = ioh.get_problem(fid=24,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 25
p25 = ioh.get_problem(fid=25,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue]

# A list of the whole problems 
Problems: list[ioh.problem.PBO] = [p1, p2, p3, p18, p23, p24, p25]

In [47]:
# create a list of 3 loggers for each algorithm -- Random Search, RLS (random local search) and (1+1)-EA
root_folder = "my-experiment"  # pyright: ignore[reportArgumentType]
if os.path.exists(root_folder):
    rmtree(root_folder, ignore_errors=True)

folder_name: str = "run"
algorithm_names: list[str] = ["RandomSearch", "RLS", "OnePlusOneEA"]
# info should be a short description of the algorithm
algorithm_info = [
    "Random Search Algorithm",
    "Random Local Search Algorithm",
    "(1+1)-EA Algorithm"
]

# for name in algorithm_names:

In [48]:
budget = num_iterations = 100000 # budget is the number of function evaluations

In [53]:
Loggers: list[logger.Analyzer] = []
# create a list of loggers for each algorithm
for i, name in enumerate(algorithm_names):
    Loggers.append(
        logger.Analyzer(
            root=root_folder,  # pyright: ignore[reportArgumentType]
            folder_name=folder_name,
            algorithm_name=name,
            algorithm_info=algorithm_info[i]
        )
    )

L_random_search = Loggers[0]
L_RLS = Loggers[1]
L_OnePlusOneEA = Loggers[2]

#### testing randoms search on problem 1

In [ ]:
num_experiments: int = 10 
p1.attach_logger(L_random_search)
algorithm = RandomSearch(budget=budget)
run_experiment(problem=p1, 
            algorithm=algorithm, 
            num_runs=num_experiments)


### Run random search on all our problems (fingers crossed!)

In [58]:
for problem in Problems:
    L_random_search = Loggers[0]
    print(f"========================================")
    # for logger_instance in Loggers:
    #     problem.attach_logger(logger_instance)
    problem.attach_logger(L_random_search)
    # run random search
    run_experiment(problem=problem, 
                algorithm=RandomSearch(budget=budget), 
                num_runs=num_experiments)
    print("========================================")
    del L_random_search

100.0
run: 1 - best found:  73.000
run: 2 - best found:  72.000
run: 3 - best found:  71.000
run: 4 - best found:  71.000
run: 5 - best found:  71.000
run: 6 - best found:  76.000
run: 7 - best found:  71.000
run: 8 - best found:  71.000
run: 9 - best found:  70.000
run: 10 - best found:  71.000
100.0
run: 1 - best found:  17.000
run: 2 - best found:  17.000
run: 3 - best found:  18.000
run: 4 - best found:  17.000
run: 5 - best found:  20.000
run: 6 - best found:  16.000
run: 7 - best found:  19.000
run: 8 - best found:  17.000
run: 9 - best found:  18.000
run: 10 - best found:  23.000
5050.0
run: 1 - best found:  3737.000
run: 2 - best found:  3678.000
run: 3 - best found:  3674.000
run: 4 - best found:  3790.000
run: 5 - best found:  3722.000
run: 6 - best found:  3705.000
run: 7 - best found:  3773.000
run: 8 - best found:  3748.000
run: 9 - best found:  3763.000
run: 10 - best found:  3802.000
inf
run: 1 - best found:  2.430
run: 2 - best found:  2.347
run: 3 - best found:  2.411
